## Exploring the geographic locations

In [16]:
import folium
from rdflib import Graph

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [17]:
# Create a Graph
g = Graph().parse("rdf/movingImageArchiveEnriched.rdf")

In [18]:
print('##### schema:Place')

# Query the data in g using SPARQL
# This query returns the 'name' of all ``schema:Place`` instances
q = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX schema: <http://schema.org/>
    PREFIX wgs: <http://www.w3.org/2003/01/geo/wgs84_pos#>

    SELECT distinct ?p ?lat ?long ?lbl ?wikidata ?geonames
    WHERE {
        ?p rdf:type schema:Place .
        ?p skos:prefLabel ?lbl .
        ?p wgs:long ?long .
        ?p wgs:lat ?lat .
        ?p owl:sameAs ?wikidata . FILTER ( strstarts(str(?wikidata), "https://www.wikidata.org/wiki/") ).
        ?p owl:sameAs ?geonames . FILTER ( strstarts(str(?geonames), "https://www.geonames.org/") )
    }
"""

##### schema:Place


In [19]:
# Apply the query to the graph and iterate through results
map_circles = folium.Map(location=[55.86,-4.25], tiles="OpenStreetMap", zoom_start=5)

for r in g.query(q):
    idwikidata = r['wikidata']
    lat = r['lat']
    lon = r['long']
    idgeonames = r['geonames']
    label = r['lbl']
    
    text_popup = "Records in <a href='" + idwikidata + "'>" + label + "</a>"

    folium.Circle(
      location=[lat, lon],
      popup=text_popup,
      #radius=float(total)/10,
      color='crimson',
      fill=True,
      fill_color='crimson'
    ).add_to(map_circles)

map_circles

In [20]:
print('##### schema:Place')

# Query the data in g using SPARQL
# This query returns the 'name' of all ``schema:Place`` instances
q = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX schema: <http://schema.org/>
    PREFIX wgs: <http://www.w3.org/2003/01/geo/wgs84_pos#>

    SELECT distinct ?wikidata
    WHERE {
        ?p rdf:type schema:Place .
        ?p owl:sameAs ?wikidata . FILTER ( strstarts(str(?wikidata), "https://www.wikidata.org/wiki/") ).
    }
"""

##### schema:Place


In [21]:
for r in g.query(q):
    idwikidata = r['wikidata']
    print(idwikidata)

https://www.wikidata.org/wiki/Q793283
https://www.wikidata.org/wiki/Q207257
https://www.wikidata.org/wiki/Q211091
https://www.wikidata.org/wiki/Q980084
https://www.wikidata.org/wiki/Q17582129
https://www.wikidata.org/wiki/Q1247435
https://www.wikidata.org/wiki/Q652539
https://www.wikidata.org/wiki/Q2421
https://www.wikidata.org/wiki/Q23436
https://www.wikidata.org/wiki/Q1061313
https://www.wikidata.org/wiki/Q189912
https://www.wikidata.org/wiki/Q530296
https://www.wikidata.org/wiki/Q81052
https://www.wikidata.org/wiki/Q202177
https://www.wikidata.org/wiki/Q54809
https://www.wikidata.org/wiki/Q786649
https://www.wikidata.org/wiki/Q664892
https://www.wikidata.org/wiki/Q1247396
https://www.wikidata.org/wiki/Q1147435
https://www.wikidata.org/wiki/Q9177476
https://www.wikidata.org/wiki/Q47134
https://www.wikidata.org/wiki/Q3643362
https://www.wikidata.org/wiki/Q4093
https://www.wikidata.org/wiki/Q206934
https://www.wikidata.org/wiki/Q550606
https://www.wikidata.org/wiki/Q864668
https://www.

In [22]:
from IPython.display import IFrame

IFrame(src='https://w.wiki/5o94', width=900, height=700)